In [1]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import StandardScaler

### Load intermediate layer output

In [2]:
in_data = np.load('data/dnn_base/in_data.npy')

layer1_output = np.load('data/dnn_base/layer1_output.npy')
layer2_output = np.load('data/dnn_base/layer2_output.npy')
layer3_output = np.load('data/dnn_base/layer3_output.npy')
layer4_output = np.load('data/dnn_base/layer4_output.npy')
layer5_output = np.load('data/dnn_base/layer5_output.npy')

activation1_output = np.load('data/dnn_base/activation1_output.npy')
activation2_output = np.load('data/dnn_base/activation2_output.npy')
activation3_output = np.load('data/dnn_base/activation3_output.npy')
activation4_output = np.load('data/dnn_base/activation4_output.npy')
activation5_output = np.load('data/dnn_base/activation5_output.npy')

In [3]:
print()
print("Model intermediate layer output")
print("==========================")
print()
print("in_data :", in_data.shape)
print("----------------------------------------------")
print("layer1_output :", layer1_output.shape)
print("----------------------------------------------")
print("activation1_output :", activation1_output.shape)
print("----------------------------------------------")
print("layer2_output :", layer2_output.shape)
print("----------------------------------------------")
print("activation2_output :", activation2_output.shape)
print("----------------------------------------------")
print("layer3_output :", layer3_output.shape)
print("----------------------------------------------")
print("activation3_output :", activation3_output.shape)
print("----------------------------------------------")
print("layer4_output :", layer4_output.shape)
print("----------------------------------------------")
print("activation4_output :", activation4_output.shape)
print("----------------------------------------------")
print("layer5_output :", layer5_output.shape)
print("----------------------------------------------")
print("activation5_output :", activation5_output.shape) # <------- softmax func


Model intermediate layer output

in_data : (10000, 784)
----------------------------------------------
layer1_output : (10000, 512)
----------------------------------------------
activation1_output : (10000, 512)
----------------------------------------------
layer2_output : (10000, 256)
----------------------------------------------
activation2_output : (10000, 256)
----------------------------------------------
layer3_output : (10000, 128)
----------------------------------------------
activation3_output : (10000, 128)
----------------------------------------------
layer4_output : (10000, 64)
----------------------------------------------
activation4_output : (10000, 64)
----------------------------------------------
layer5_output : (10000, 10)
----------------------------------------------
activation5_output : (10000, 10)


### Kernel Tools

In [12]:
class RBFkernel():
    def __init__(self, sigma=0.5):
        self.sigma = sigma
        
    def __call__(self, x, y):
        numerator = -1 * np.linalg.norm(x - y, ord=2)**2
        denominator = 2 * (self.sigma**2)
        return np.exp(numerator / denominator)
    
    def get_params(self):
        return self.sigma
    
    def set_params(self, sigma):
        self.sigma = sigma
                
            
def gram_matrix(kernel, data, m):
    """
    Arguments:
    =========
    - kernel : kernel function 
    - data : data samples, shape=(m, dim(data_i))
    - m : number of samples
    """
    gram_matrix = np.zeros((m, m))
    for i in tqdm(range(m)):
        for j in range(m):
            gram_matrix[i][j] = kernel(data[i], data[j])
            
    return gram_matrix


def hsic(k, l, m, X, Y):
    """
    Arguments:
    =========
    - k : kernel function for X
    - l : kernel function for Y
    - m : number of samples
    - X : data samples, shape=(m, dim(X_i))
    - Y : data samples, shape=(m, dim(Y_i))
    """
    H = np.full((m, m), -(1/m)) + np.eye(m)
    K = gram_matrix(k, X, m)
    print("Gram(X) :", K, "\nGram(X) mean :", K.mean())
    L = gram_matrix(l, Y, m)
    print("Gram(Y) :", L, "\nGram(Y) mean :", L.mean())
    HSIC = np.trace(np.dot(K, np.dot(H, np.dot(L, H)))) / ((m - 1)**2)
    return HSIC

**sample code**

```python
kernel = RBFkernel(sigma=1)
kernel(np.array([1,2,3]), np.array([1,2,6]))
```
Output: 0.011108996538242306

### Measuring HSIC

In [79]:
# In_data vs layer1_output
# 40 min

m=10000
# ----------------------------
X = in_data
Y = in_data
# ----------------------------

print("X.shape :", X.shape)
print("Y.shape :", Y.shape)

kernel_x = RBFkernel(sigma=75)
kernel_y = RBFkernel(sigma=75 * np.sqrt(X.shape[1]/Y.shape[1]))

hsic_value = hsic(kernel_x, kernel_y, m, X, Y)
print("HSIC =", hsic_value)

X.shape : (10000, 784)
Y.shape : (10000, 784)


KeyboardInterrupt: 

In [5]:
# In_data vs layer1_output
# 40 min

# ----------------------------
X = in_data
Y = layer1_output
# ----------------------------

m =10000
print("X.shape :", X.shape)
print("Y.shape :", Y.shape)

kernel_x = RBFkernel(sigma=75)
kernel_y = RBFkernel(sigma=75 * np.sqrt(X.shape[1]/Y.shape[1]))

hsic_value = hsic(kernel_x, kernel_y, m, X, Y)
print("HSIC =", hsic_value)

X.shape : (10000, 784)
Y.shape : (10000, 512)



GramMatrix : [[1.         0.9887758  0.9935449  ... 0.99294203 0.99180394 0.98631674]
 [0.9887758  1.         0.99145562 ... 0.98640696 0.99069777 0.98744565]
 [0.9935449  0.99145562 1.         ... 0.99248119 0.99297051 0.98807993]
 ...
 [0.99294203 0.98640696 0.99248119 ... 1.         0.99240353 0.98850993]
 [0.99180394 0.99069777 0.99297051 ... 0.99240353 1.         0.98790197]
 [0.98631674 0.98744565 0.98807993 ... 0.98850993 0.98790197 1.        ]]
GramMatrix mean : 0.9906519159899633



GramMatrix : [[1.         0.29767587 0.5740685  ... 0.55433925 0.49301655 0.22334681]
 [0.29767587 1.         0.39467914 ... 0.22619553 0.33865132 0.32410016]
 [0.5740685  0.39467914 1.         ... 0.43806588 0.4842447  0.22078869]
 ...
 [0.55433925 0.22619553 0.43806588 ... 1.         0.51134009 0.31106015]
 [0.49301655 0.33865132 0.4842447  ... 0.51134009 1.         0.27334086]
 [0.22334681 0.32410016 0.22078869 ... 0.31106015 0.27334086 1.        ]]
GramMatrix mean : 0.4446806396698412
HSIC = 0.0001690372894172779


## TODO: gram_matrix: K,Lを導出後に，data/calc_hsic/以下へ.npyで保存する